
Welcome to this tutorial! Here, we'll use **Qiskit v1.4** to:
1. Build a **Bell state** quantum circuit  
2. Simulate the quantum circuit  
3. Analyze the results  

---
###  What is a Bell State?
Recall the Bell state, one of the **simplest maximally entangled states**.When measuring one qubit in the Bell state, the outcome is a random (50% |0⟩, 50% |1⟩), but once measured, the other qubit’s state is instantly determined due to quantum entanglement, no matter how far apart they are.

Let's implement it in Qiskit! We first import our dependecies. We import the very specifics we need here for this to be light weight. 


In [ ]:
import numpy as np 
    #needed for numerical operations

from qiskit import QuantumCircuit, transpile, assemble
    # QuantumCircuit: Defines and manipulates quantum circuits.  
    # transpile: Optimizes quantum circuits for a specific quantum backend.  
    # assemble: Converts the circuit into a runnable format for execution on simulators or real quantum hardware.

from qiskit.primitives import StatevectorSampler    
    # A high-level Qiskit primitive that allows simulation of quantum circuits using state vectors.

from qiskit.visualization import plot_histogram
    # Lets us visualize quantum measurement results in a histogram format.
from qiskit_aer import Aer  
    # Provides high-performance simulators to run quantum circuits on classical hardware.
import matplotlib.pyplot as plt  
    # For plotting

## Step 1: Creating a Quantum Circuit
We create a 2-qubit quantum circuit and apply:

    1. Hadamard Gate (H): Places qubit 0 in superposition  
    2. CNOT Gate (CX): Entangles qubit 0 with qubit 1  


In [ ]:
# Create a quantum circuit with 2 qubits
qc = QuantumCircuit(2)

# Apply a Hadamard gate to qubit 0 (superposition)
qc.h(0)

# Apply a CNOT gate to entangle qubits 0 and 1
qc.cx(0, 1)

# several ways to visualise the quantum circuit, like:
print(qc)             # 1. Prints ascii representation to the terminal, as QuantumCircuit has a __str__ method
qc.draw('mpl')        # 2. Produces a matplotlib diagram. draw("text") would print an ascii representation as well. 


##  Step 2: Transpile and Choose a backend. 
At this stage, we have **built a quantum circuit**, but it has **not been executed** yet. It is siply an **abstract representation** of quantum operations.

Before running it on real hardware or a simulator, we need to:

    1. Transpile the circuit  
    2. Choose a quantum backend
    3. Prepare the circuit for execution


### Transpilation
Transpilation is like "translating" your quantum circuit so that it can run correctly on the backend you choose. A quantum computer has **physical constraints**, so depending on the computer, certain operations (gates) will not be natively supported. **`transpile()`** converts the circuit into an **optimized** version that respects the **hardware's connectivity** and **gate set**.

Since we are simulating on a classical computer, we don’t have to worry about real hardware limitations, and the circuit would still execute without it, but we still transpile because: 

    1. It optimizes the circuit for faster and more efficient execution.
    2. It removes unnecessary operations that don’t affect the final result.
    3. It ensures compatibility in case we later want to run on a real quantum device.

### Choosing a backend
- Qiskit provides multiple quantum **simulators** (simulated quantum computers), like aer_simuator, qasm_simulator and statevector_simulator
- We use **`'aer_simulator'`**, which beacuse it is efficient and allows us to simulate both noise-free and noisy environments.


In [ ]:
# Choose a simulator backend
simulator = Aer.get_backend('aer_simulator')

# Transpile the circuit for the simulator
qc_transpiled = transpile(qc, simulator)
print("Transpiled Circuit Ready for Execution:")
print(qc_transpiled)

##  Step 3: Running the Circuit and Getting Results
Now that the circuit is **transpiled** and a simulator is chosen, we can **execute** it.

### What Happens During Execution?
1. Ensure the circuit includes measurement gates
    * A quantum circuit does not measure qubits by default.
    * We explicitly add measurement gates using qc.measure_all(inplace=False) to store results in classical registers after execution.
2. Send the circuit to the quantum backend for execution.
    * In most quantum simulations and real hardware runs, shots controls the number of times the circuit is executed.
    * If shots=1024, circuit is executed 1024 times and results are statistically sampled. 


In [ ]:
# Add measurement, inplace=False creates a new circuit with measurement gates instead of modifying qc directly
qc_measured = qc.measure_all(inplace=False)  

# Run the circuit on the simulator        
job = sampler.run([qc_measured], shots=1024)  
 
# Get and print results 
result = job.result()
print(result.get_counts())


## Congratulations! 
You got your first quantum program to run! You have all the power in your hands now. Go wild and expirament. 

You should have a result similar to {'00': 510, '11': 514}. This means that the circuit collapsed to the state |00⟩ 510 times out of 1024, and |11⟩ 514 times out of 1024. Such results suggest that our quantum circuit created an equal superposition of (a 50/50 chance of either) |00⟩ and |11⟩ before measurement. 

#### **Why Are `|01⟩` and `|10⟩` Missing?**
If the quantum circuit entangled two qubits into a **Bell state**, the system is described by:
$
\frac{|00\rangle + |11\rangle}{\sqrt{2}}
$


This means that the two qubits are **correlated**—measuring one qubit immediately determine~s the state of the other.

- If **`q_0 = 0`**, then **`q_1` must also be `0`**, resulting in **`|00⟩`**.
- If **`q_0 = 1`**, then **`q_1` must also be `1`**, resulting in **`|11⟩`**.

Since the system **never collapses into `|01⟩` or `|10⟩`**, those states do not appear in the measurement results.